In [1]:
import os
import sys
from importnb import Notebook
from typing import Optional, Union, List, Callable
from io import BytesIO
import struct
import numpy as np
import pandas as pd
import re

PKG_ROOT = os.path.dirname(os.path.realpath(os.getcwd()))
if not PKG_ROOT in sys.path:
    sys.path.append(PKG_ROOT)

In [2]:
class Edit2DA:
    
    def __init__(self, name:str):
        self.source = os.path.join(PKG_ROOT, 'resources/2da', name +'.2da')
        self.target = os.path.join(PKG_ROOT, 'resources/modified', name +'.2da')
        self.csv = os.path.join(PKG_ROOT, 'resources/modified', name +'.csv')
        
    def from_str(self, input:str):
        with open(self.source, 'w') as s:
            s.write(input)      
            
    def load(self):
        with open(self.source, 'r') as s:
            f = s.read()
        self.text = f.split('\n')
        self.df = pd.DataFrame([_.split() for _ in self.text[3:]], columns=[''] + self.text[2].split())
        self.default = self.text[1]
        self.header = self.text[0]
        self.df.to_csv(self.csv)
        
    def load_csv(self):
        self.df = pd.read_csv(self.csv, index_col=0).rename(columns={'Unnamed: 1': ''})
    
    def save(self, from_csv:bool=True):
        if from_csv:
            self.load_csv()
        text = [self.header, self.default, ' '.join(self.df.columns)] + [' '.join(_) for _ in self.df.astype(str).values]
        with open(self.target, 'w') as s:
            s.write('\n'.join(text))    
    
              

In [93]:
def parse_list(filename:str, pattern:re.Pattern=re.compile(r'(?P<ItemCode>[a-z0-9_]+)\s+-\s+(?P<Item>.*)')):
    with open(filename, 'r') as s:
        c = s.read()
    cols = list(pattern.groupindex.keys())
    df = pd.DataFrame(re.findall(pattern, c), columns=cols)
    df[cols[1]] = df[cols[1]].apply(lambda x: re.sub('_+', '_', re.sub(r'[^a-zA-Z0-9]', '_', x.replace('+', '_PLUS').replace("'s", ''))).strip().strip('_').upper())
    df = df.drop_duplicates().groupby(cols[1]).apply(lambda x:x.reset_index(drop=True)).reset_index(level=1)
    df[cols[1]] = df.apply(lambda x: x[cols[1]] if x['level_1']==0 else x[cols[1]] + '_' + str(x['level_1']), axis=1)
    return df[cols].reset_index(drop=True)
    
    

In [72]:
d = Edit2DA('allfeats')